In [7]:
import polars as pl
import torch
import numpy as np
import os

DATA_PATH = "../data/processed"


In [8]:
# load data
train = pl.read_parquet(os.path.join(DATA_PATH, "train.parquet"))
test = pl.read_parquet(os.path.join(DATA_PATH, "test.parquet"))

In [ ]:
# train pytorch rnn model predicting whether it will rain the next day or not on time series data.
# use the value of rainfall in the last 7 days along with location as input features, and whether 
# it rained the next day as the target variable.
class RainfallRNN(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RainfallRNN, self).__init__()
        self.rnn = torch.nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = torch.nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.rnn(x)
        out = self.fc(out[:, -1, :])
        return torch.sigmoid(out)


In [11]:
# prepare data for training
drop_cols = ["date", "station_name", "y"]
X_train = train.drop(drop_cols).to_numpy()
X_test = test.drop(drop_cols).to_numpy()

y_train = train["y"].to_numpy().reshape(-1, 1)
y_test = test["y"].to_numpy().reshape(-1, 1)

In [ ]:
# initiate model, loss function and optimizer
model = RainfallRNN(input_size=X_train.shape[0], hidden_size=64, output_size=1)
criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# train model
